In [ ]:
import sys

import pprint as pp

sys.path.insert(0, "/home/peter/Documents/DataSet/")

import source.db as db

client = db.connect("localhost", 27017)

equivalent = client['refactoring']['equivalent']

count = equivalent.count_documents({})


In [ ]:
import difflib

file1_content = """This is the content of file 1.
It has some differences compared to file 2.
"""

file2_content = """This is the content of file 2.
It has some differences compared to file 1.
"""

# split the strings into lines
file1_lines = file1_content.splitlines()
file2_lines = file2_content.splitlines()

# use difflib to get the differences
differ = difflib.Differ()
diff = list(differ.compare(file1_lines, file2_lines))

# print the differences
for line in diff: print(line)


In [ ]:
record = {
  "_id": "bce710c0548bc7a7d95c683dc4ee2a1e08ab41d5a20ae2d78695b5d73b8d8ed3",
  "autopep": "class Graph:\n    \"\"\"\n    Data structure to store graphs (based on adjacency lists)\n    \"\"\"\n\n    def __init__(self):\n        self.num_vertices = 0\n        self.num_edges = 0\n        self.adjacency = {}\n\n    def add_vertex(self, vertex):\n        \"\"\"\n        Adds a vertex to the graph\n\n        \"\"\"\n        if vertex not in self.adjacency:\n            self.adjacency[vertex] = {}\n            self.num_vertices += 1\n\n    def add_edge(self, head, tail, weight):\n        \"\"\"\n        Adds an edge to the graph\n\n        \"\"\"\n\n        self.add_vertex(head)\n        self.add_vertex(tail)\n\n        if head == tail:\n            return\n\n        self.adjacency[head][tail] = weight\n        self.adjacency[tail][head] = weight\n\n    def distinct_weight(self):\n        \"\"\"\n        For Boruvks's algorithm the weights should be distinct\n        Converts the weights to be distinct\n\n        \"\"\"\n        edges = self.get_edges()\n        for edge in edges:\n            head, tail, weight = edge\n            edges.remove((tail, head, weight))\n        for i in range(len(edges)):\n            edges[i] = list(edges[i])\n\n        edges.sort(key=lambda e: e[2])\n        for i in range(len(edges) - 1):\n            if edges[i][2] >= edges[i + 1][2]:\n                edges[i + 1][2] = edges[i][2] + 1\n        for edge in edges:\n            head, tail, weight = edge\n            self.adjacency[head][tail] = weight\n            self.adjacency[tail][head] = weight\n\n    def __str__(self):\n        \"\"\"\n        Returns string representation of the graph\n        \"\"\"\n        string = \"\"\n        for tail in self.adjacency:\n            for head in self.adjacency[tail]:\n                weight = self.adjacency[head][tail]\n                string += f\"{head} -> {tail} == {weight}\\n\"\n        return string.rstrip(\"\\n\")\n\n    def get_edges(self):\n        \"\"\"\n        Returna all edges in the graph\n        \"\"\"\n        output = []\n        for tail in self.adjacency:\n            for head in self.adjacency[tail]:\n                output.append((tail, head, self.adjacency[head][tail]))\n        return output\n\n    def get_vertices(self):\n        \"\"\"\n        Returns all vertices in the graph\n        \"\"\"\n        return self.adjacency.keys()\n\n    @staticmethod\n    def build(vertices=None, edges=None):\n        \"\"\"\n        Builds a graph from the given set of vertices and edges\n\n        \"\"\"\n        g = Graph()\n        if vertices is None:\n            vertices = []\n        if edges is None:\n            edge = []\n        for vertex in vertices:\n            g.add_vertex(vertex)\n        for edge in edges:\n            g.add_edge(*edge)\n        return g\n\n    class UnionFind:\n        \"\"\"\n        Disjoint set Union and Find for Boruvka's algorithm\n        \"\"\"\n\n        def __init__(self):\n            self.parent = {}\n            self.rank = {}\n\n        def __len__(self):\n            return len(self.parent)\n\n        def make_set(self, item):\n            if item in self.parent:\n                return self.find(item)\n\n            self.parent[item] = item\n            self.rank[item] = 0\n            return item\n\n        def find(self, item):\n            if item not in self.parent:\n                return self.make_set(item)\n            if item != self.parent[item]:\n                self.parent[item] = self.find(self.parent[item])\n            return self.parent[item]\n\n        def union(self, item1, item2):\n            root1 = self.find(item1)\n            root2 = self.find(item2)\n\n            if root1 == root2:\n                return root1\n\n            if self.rank[root1] > self.rank[root2]:\n                self.parent[root2] = root1\n                return root1\n\n            if self.rank[root1] < self.rank[root2]:\n                self.parent[root1] = root2\n                return root2\n\n            if self.rank[root1] == self.rank[root2]:\n                self.rank[root1] += 1\n                self.parent[root2] = root1\n                return root1\n            return None\n\n    @staticmethod\n    def boruvka_mst(graph):\n        \"\"\"\n        Implementation of Boruvka's algorithm\n        >>> g = Graph()\n        >>> g = Graph.build([0, 1, 2, 3], [[0, 1, 1], [0, 2, 1],[2, 3, 1]])\n        >>> g.distinct_weight()\n        >>> bg = Graph.boruvka_mst(g)\n        >>> print(bg)\n        1 -> 0 == 1\n        2 -> 0 == 2\n        0 -> 1 == 1\n        0 -> 2 == 2\n        3 -> 2 == 3\n        2 -> 3 == 3\n        \"\"\"\n        num_components = graph.num_vertices\n\n        union_find = Graph.UnionFind()\n        mst_edges = []\n        while num_components > 1:\n            cheap_edge = {}\n            for vertex in graph.get_vertices():\n                cheap_edge[vertex] = -1\n\n            edges = graph.get_edges()\n            for edge in edges:\n                head, tail, weight = edge\n                edges.remove((tail, head, weight))\n            for edge in edges:\n                head, tail, weight = edge\n                set1 = union_find.find(head)\n                set2 = union_find.find(tail)\n                if set1 != set2:\n                    if cheap_edge[set1] == -1 or cheap_edge[set1][2] > weight:\n                        cheap_edge[set1] = [head, tail, weight]\n\n                    if cheap_edge[set2] == -1 or cheap_edge[set2][2] > weight:\n                        cheap_edge[set2] = [head, tail, weight]\n            for vertex in cheap_edge:\n                if cheap_edge[vertex] != -1:\n                    head, tail, weight = cheap_edge[vertex]\n                    if union_find.find(head) != union_find.find(tail):\n                        union_find.union(head, tail)\n                        mst_edges.append(cheap_edge[vertex])\n                        num_components = num_components - 1\n        mst = Graph.build(edges=mst_edges)\n        return mst\n",
  "autopep+for_to_comp": "class Graph:\n    \"\"\"\n    Data structure to store graphs (based on adjacency lists)\n    \"\"\"\n\n    def __init__(self):\n        self.num_vertices = 0\n        self.num_edges = 0\n        self.adjacency = {}\n\n    def add_vertex(self, vertex):\n        \"\"\"\n        Adds a vertex to the graph\n\n        \"\"\"\n        if vertex not in self.adjacency:\n            self.adjacency[vertex] = {}\n            self.num_vertices += 1\n\n    def add_edge(self, head, tail, weight):\n        \"\"\"\n        Adds an edge to the graph\n\n        \"\"\"\n        self.add_vertex(head)\n        self.add_vertex(tail)\n        if head == tail:\n            return\n        self.adjacency[head][tail] = weight\n        self.adjacency[tail][head] = weight\n\n    def distinct_weight(self):\n        \"\"\"\n        For Boruvks's algorithm the weights should be distinct\n        Converts the weights to be distinct\n\n        \"\"\"\n        edges = self.get_edges()\n        for edge in edges:\n            (head, tail, weight) = edge\n            edges.remove((tail, head, weight))\n        for i in range(len(edges)):\n            edges[i] = list(edges[i])\n        edges.sort(key=lambda e: e[2])\n        for i in range(len(edges) - 1):\n            if edges[i][2] >= edges[i + 1][2]:\n                edges[i + 1][2] = edges[i][2] + 1\n        for edge in edges:\n            (head, tail, weight) = edge\n            self.adjacency[head][tail] = weight\n            self.adjacency[tail][head] = weight\n\n    def __str__(self):\n        \"\"\"\n        Returns string representation of the graph\n        \"\"\"\n        string = ''\n        for tail in self.adjacency:\n            for head in self.adjacency[tail]:\n                weight = self.adjacency[head][tail]\n                string += f'{head} -> {tail} == {weight}\\n'\n        return string.rstrip('\\n')\n\n    def get_edges(self):\n        \"\"\"\n        Returna all edges in the graph\n        \"\"\"\n        output = []\n        for tail in self.adjacency:\n            for head in self.adjacency[tail]:\n                output.append((tail, head, self.adjacency[head][tail]))\n        return output\n\n    def get_vertices(self):\n        \"\"\"\n        Returns all vertices in the graph\n        \"\"\"\n        return self.adjacency.keys()\n\n    @staticmethod\n    def build(vertices=None, edges=None):\n        \"\"\"\n        Builds a graph from the given set of vertices and edges\n\n        \"\"\"\n        g = Graph()\n        if vertices is None:\n            vertices = []\n        if edges is None:\n            edge = []\n        for vertex in vertices:\n            g.add_vertex(vertex)\n        for edge in edges:\n            g.add_edge(*edge)\n        return g\n\n    class UnionFind:\n        \"\"\"\n        Disjoint set Union and Find for Boruvka's algorithm\n        \"\"\"\n\n        def __init__(self):\n            self.parent = {}\n            self.rank = {}\n\n        def __len__(self):\n            return len(self.parent)\n\n        def make_set(self, item):\n            if item in self.parent:\n                return self.find(item)\n            self.parent[item] = item\n            self.rank[item] = 0\n            return item\n\n        def find(self, item):\n            if item not in self.parent:\n                return self.make_set(item)\n            if item != self.parent[item]:\n                self.parent[item] = self.find(self.parent[item])\n            return self.parent[item]\n\n        def union(self, item1, item2):\n            root1 = self.find(item1)\n            root2 = self.find(item2)\n            if root1 == root2:\n                return root1\n            if self.rank[root1] > self.rank[root2]:\n                self.parent[root2] = root1\n                return root1\n            if self.rank[root1] < self.rank[root2]:\n                self.parent[root1] = root2\n                return root2\n            if self.rank[root1] == self.rank[root2]:\n                self.rank[root1] += 1\n                self.parent[root2] = root1\n                return root1\n            return None\n\n    @staticmethod\n    def boruvka_mst(graph):\n        \"\"\"\n        Implementation of Boruvka's algorithm\n        >>> g = Graph()\n        >>> g = Graph.build([0, 1, 2, 3], [[0, 1, 1], [0, 2, 1],[2, 3, 1]])\n        >>> g.distinct_weight()\n        >>> bg = Graph.boruvka_mst(g)\n        >>> print(bg)\n        1 -> 0 == 1\n        2 -> 0 == 2\n        0 -> 1 == 1\n        0 -> 2 == 2\n        3 -> 2 == 3\n        2 -> 3 == 3\n        \"\"\"\n        num_components = graph.num_vertices\n        union_find = Graph.UnionFind()\n        mst_edges = []\n        while num_components > 1:\n            cheap_edge = {vertex: -1 for vertex in graph.get_vertices()}\n            edges = graph.get_edges()\n            for edge in edges:\n                (head, tail, weight) = edge\n                edges.remove((tail, head, weight))\n            for edge in edges:\n                (head, tail, weight) = edge\n                set1 = union_find.find(head)\n                set2 = union_find.find(tail)\n                if set1 != set2:\n                    if cheap_edge[set1] == -1 or cheap_edge[set1][2] > weight:\n                        cheap_edge[set1] = [head, tail, weight]\n                    if cheap_edge[set2] == -1 or cheap_edge[set2][2] > weight:\n                        cheap_edge[set2] = [head, tail, weight]\n            for vertex in cheap_edge:\n                if cheap_edge[vertex] != -1:\n                    (head, tail, weight) = cheap_edge[vertex]\n                    if union_find.find(head) != union_find.find(tail):\n                        union_find.union(head, tail)\n                        mst_edges.append(cheap_edge[vertex])\n                        num_components = num_components - 1\n        mst = Graph.build(edges=mst_edges)\n        return mst\n",
  "autopep+ast_all": "class Graph:\n    \"\"\"\n    Data structure to store graphs (based on adjacency lists)\n    \"\"\"\n\n    def __init__(self):\n        self.num_vertices = 0\n        self.num_edges = 0\n        self.adjacency = {}\n\n    def add_vertex(self, vertex):\n        \"\"\"\n        Adds a vertex to the graph\n\n        \"\"\"\n        if vertex not in self.adjacency:\n            self.adjacency[vertex] = {}\n            self.num_vertices += 1\n\n    def add_edge(self, head, tail, weight):\n        \"\"\"\n        Adds an edge to the graph\n\n        \"\"\"\n        self.add_vertex(head)\n        self.add_vertex(tail)\n        if head == tail:\n            return\n        self.adjacency[head][tail] = weight\n        self.adjacency[tail][head] = weight\n\n    def distinct_weight(self):\n        \"\"\"\n        For Boruvks's algorithm the weights should be distinct\n        Converts the weights to be distinct\n\n        \"\"\"\n        edges = self.get_edges()\n        for edge in edges:\n            (head, tail, weight) = edge\n            edges.remove((tail, head, weight))\n        for i in range(len(edges)):\n            edges[i] = list(edges[i])\n        edges.sort(key=lambda e: e[2])\n        for i in range(len(edges) - 1):\n            if edges[i][2] >= edges[i + 1][2]:\n                edges[i + 1][2] = edges[i][2] + 1\n        for edge in edges:\n            (head, tail, weight) = edge\n            self.adjacency[head][tail] = weight\n            self.adjacency[tail][head] = weight\n\n    def __str__(self):\n        \"\"\"\n        Returns string representation of the graph\n        \"\"\"\n        string = ''\n        for tail in self.adjacency:\n            for head in self.adjacency[tail]:\n                weight = self.adjacency[head][tail]\n                string += f'{head} -> {tail} == {weight}\\n'\n        return string.rstrip('\\n')\n\n    def get_edges(self):\n        \"\"\"\n        Returna all edges in the graph\n        \"\"\"\n        output = []\n        for tail in self.adjacency:\n            for head in self.adjacency[tail]:\n                output.append((tail, head, self.adjacency[head][tail]))\n        return output\n\n    def get_vertices(self):\n        \"\"\"\n        Returns all vertices in the graph\n        \"\"\"\n        return self.adjacency.keys()\n\n    @staticmethod\n    def build(vertices=None, edges=None):\n        \"\"\"\n        Builds a graph from the given set of vertices and edges\n\n        \"\"\"\n        g = Graph()\n        if vertices is None:\n            vertices = []\n        if edges is None:\n            edge = []\n        for vertex in vertices:\n            g.add_vertex(vertex)\n        for edge in edges:\n            g.add_edge(*edge)\n        return g\n\n    class UnionFind:\n        \"\"\"\n        Disjoint set Union and Find for Boruvka's algorithm\n        \"\"\"\n\n        def __init__(self):\n            self.parent = {}\n            self.rank = {}\n\n        def __len__(self):\n            return len(self.parent)\n\n        def make_set(self, item):\n            if item in self.parent:\n                return self.find(item)\n            self.parent[item] = item\n            self.rank[item] = 0\n            return item\n\n        def find(self, item):\n            if item not in self.parent:\n                return self.make_set(item)\n            if item != self.parent[item]:\n                self.parent[item] = self.find(self.parent[item])\n            return self.parent[item]\n\n        def union(self, item1, item2):\n            root1 = self.find(item1)\n            root2 = self.find(item2)\n            if root1 == root2:\n                return root1\n            if self.rank[root1] > self.rank[root2]:\n                self.parent[root2] = root1\n                return root1\n            if self.rank[root1] < self.rank[root2]:\n                self.parent[root1] = root2\n                return root2\n            if self.rank[root1] == self.rank[root2]:\n                self.rank[root1] += 1\n                self.parent[root2] = root1\n                return root1\n            return None\n\n    @staticmethod\n    def boruvka_mst(graph):\n        \"\"\"\n        Implementation of Boruvka's algorithm\n        >>> g = Graph()\n        >>> g = Graph.build([0, 1, 2, 3], [[0, 1, 1], [0, 2, 1],[2, 3, 1]])\n        >>> g.distinct_weight()\n        >>> bg = Graph.boruvka_mst(g)\n        >>> print(bg)\n        1 -> 0 == 1\n        2 -> 0 == 2\n        0 -> 1 == 1\n        0 -> 2 == 2\n        3 -> 2 == 3\n        2 -> 3 == 3\n        \"\"\"\n        num_components = graph.num_vertices\n        union_find = Graph.UnionFind()\n        mst_edges = []\n        while num_components > 1:\n            cheap_edge = {vertex: -1 for vertex in graph.get_vertices()}\n            edges = graph.get_edges()\n            for edge in edges:\n                (head, tail, weight) = edge\n                edges.remove((tail, head, weight))\n            for edge in edges:\n                (head, tail, weight) = edge\n                set1 = union_find.find(head)\n                set2 = union_find.find(tail)\n                if set1 != set2:\n                    if cheap_edge[set1] == -1 or cheap_edge[set1][2] > weight:\n                        cheap_edge[set1] = [head, tail, weight]\n                    if cheap_edge[set2] == -1 or cheap_edge[set2][2] > weight:\n                        cheap_edge[set2] = [head, tail, weight]\n            for vertex in cheap_edge:\n                if cheap_edge[vertex] != -1:\n                    (head, tail, weight) = cheap_edge[vertex]\n                    if union_find.find(head) != union_find.find(tail):\n                        union_find.union(head, tail)\n                        mst_edges.append(cheap_edge[vertex])\n                        num_components = num_components - 1\n        mst = Graph.build(edges=mst_edges)\n        return mst\n",
  "autopep+ast_all_but_invert_if": "class Graph:\n    \"\"\"\n    Data structure to store graphs (based on adjacency lists)\n    \"\"\"\n\n    def __init__(self):\n        self.num_vertices = 0\n        self.num_edges = 0\n        self.adjacency = {}\n\n    def add_vertex(self, vertex):\n        \"\"\"\n        Adds a vertex to the graph\n\n        \"\"\"\n        if vertex not in self.adjacency:\n            self.adjacency[vertex] = {}\n            self.num_vertices += 1\n\n    def add_edge(self, head, tail, weight):\n        \"\"\"\n        Adds an edge to the graph\n\n        \"\"\"\n        self.add_vertex(head)\n        self.add_vertex(tail)\n        if head == tail:\n            return\n        self.adjacency[head][tail] = weight\n        self.adjacency[tail][head] = weight\n\n    def distinct_weight(self):\n        \"\"\"\n        For Boruvks's algorithm the weights should be distinct\n        Converts the weights to be distinct\n\n        \"\"\"\n        edges = self.get_edges()\n        for edge in edges:\n            (head, tail, weight) = edge\n            edges.remove((tail, head, weight))\n        for i in range(len(edges)):\n            edges[i] = list(edges[i])\n        edges.sort(key=lambda e: e[2])\n        for i in range(len(edges) - 1):\n            if edges[i][2] >= edges[i + 1][2]:\n                edges[i + 1][2] = edges[i][2] + 1\n        for edge in edges:\n            (head, tail, weight) = edge\n            self.adjacency[head][tail] = weight\n            self.adjacency[tail][head] = weight\n\n    def __str__(self):\n        \"\"\"\n        Returns string representation of the graph\n        \"\"\"\n        string = ''\n        for tail in self.adjacency:\n            for head in self.adjacency[tail]:\n                weight = self.adjacency[head][tail]\n                string += f'{head} -> {tail} == {weight}\\n'\n        return string.rstrip('\\n')\n\n    def get_edges(self):\n        \"\"\"\n        Returna all edges in the graph\n        \"\"\"\n        output = []\n        for tail in self.adjacency:\n            for head in self.adjacency[tail]:\n                output.append((tail, head, self.adjacency[head][tail]))\n        return output\n\n    def get_vertices(self):\n        \"\"\"\n        Returns all vertices in the graph\n        \"\"\"\n        return self.adjacency.keys()\n\n    @staticmethod\n    def build(vertices=None, edges=None):\n        \"\"\"\n        Builds a graph from the given set of vertices and edges\n\n        \"\"\"\n        g = Graph()\n        if vertices is None:\n            vertices = []\n        if edges is None:\n            edge = []\n        for vertex in vertices:\n            g.add_vertex(vertex)\n        for edge in edges:\n            g.add_edge(*edge)\n        return g\n\n    class UnionFind:\n        \"\"\"\n        Disjoint set Union and Find for Boruvka's algorithm\n        \"\"\"\n\n        def __init__(self):\n            self.parent = {}\n            self.rank = {}\n\n        def __len__(self):\n            return len(self.parent)\n\n        def make_set(self, item):\n            if item in self.parent:\n                return self.find(item)\n            self.parent[item] = item\n            self.rank[item] = 0\n            return item\n\n        def find(self, item):\n            if item not in self.parent:\n                return self.make_set(item)\n            if item != self.parent[item]:\n                self.parent[item] = self.find(self.parent[item])\n            return self.parent[item]\n\n        def union(self, item1, item2):\n            root1 = self.find(item1)\n            root2 = self.find(item2)\n            if root1 == root2:\n                return root1\n            if self.rank[root1] > self.rank[root2]:\n                self.parent[root2] = root1\n                return root1\n            if self.rank[root1] < self.rank[root2]:\n                self.parent[root1] = root2\n                return root2\n            if self.rank[root1] == self.rank[root2]:\n                self.rank[root1] += 1\n                self.parent[root2] = root1\n                return root1\n            return None\n\n    @staticmethod\n    def boruvka_mst(graph):\n        \"\"\"\n        Implementation of Boruvka's algorithm\n        >>> g = Graph()\n        >>> g = Graph.build([0, 1, 2, 3], [[0, 1, 1], [0, 2, 1],[2, 3, 1]])\n        >>> g.distinct_weight()\n        >>> bg = Graph.boruvka_mst(g)\n        >>> print(bg)\n        1 -> 0 == 1\n        2 -> 0 == 2\n        0 -> 1 == 1\n        0 -> 2 == 2\n        3 -> 2 == 3\n        2 -> 3 == 3\n        \"\"\"\n        num_components = graph.num_vertices\n        union_find = Graph.UnionFind()\n        mst_edges = []\n        while num_components > 1:\n            cheap_edge = {vertex: -1 for vertex in graph.get_vertices()}\n            edges = graph.get_edges()\n            for edge in edges:\n                (head, tail, weight) = edge\n                edges.remove((tail, head, weight))\n            for edge in edges:\n                (head, tail, weight) = edge\n                set1 = union_find.find(head)\n                set2 = union_find.find(tail)\n                if set1 != set2:\n                    if cheap_edge[set1] == -1 or cheap_edge[set1][2] > weight:\n                        cheap_edge[set1] = [head, tail, weight]\n                    if cheap_edge[set2] == -1 or cheap_edge[set2][2] > weight:\n                        cheap_edge[set2] = [head, tail, weight]\n            for vertex in cheap_edge:\n                if cheap_edge[vertex] != -1:\n                    (head, tail, weight) = cheap_edge[vertex]\n                    if union_find.find(head) != union_find.find(tail):\n                        union_find.union(head, tail)\n                        mst_edges.append(cheap_edge[vertex])\n                        num_components = num_components - 1\n        mst = Graph.build(edges=mst_edges)\n        return mst\n",
  "_source": "class Graph:\n    \"\"\"\n    Data structure to store graphs (based on adjacency lists)\n    \"\"\"\n\n    def __init__(self):\n        self.num_vertices = 0\n        self.num_edges = 0\n        self.adjacency = {}\n\n    def add_vertex(self, vertex):\n        \"\"\"\n        Adds a vertex to the graph\n\n        \"\"\"\n        if vertex not in self.adjacency:\n            self.adjacency[vertex] = {}\n            self.num_vertices += 1\n\n    def add_edge(self, head, tail, weight):\n        \"\"\"\n        Adds an edge to the graph\n\n        \"\"\"\n\n        self.add_vertex(head)\n        self.add_vertex(tail)\n\n        if head == tail:\n            return\n\n        self.adjacency[head][tail] = weight\n        self.adjacency[tail][head] = weight\n\n    def distinct_weight(self):\n        \"\"\"\n        For Boruvks's algorithm the weights should be distinct\n        Converts the weights to be distinct\n\n        \"\"\"\n        edges = self.get_edges()\n        for edge in edges:\n            head, tail, weight = edge\n            edges.remove((tail, head, weight))\n        for i in range(len(edges)):\n            edges[i] = list(edges[i])\n\n        edges.sort(key=lambda e: e[2])\n        for i in range(len(edges) - 1):\n            if edges[i][2] >= edges[i + 1][2]:\n                edges[i + 1][2] = edges[i][2] + 1\n        for edge in edges:\n            head, tail, weight = edge\n            self.adjacency[head][tail] = weight\n            self.adjacency[tail][head] = weight\n\n    def __str__(self):\n        \"\"\"\n        Returns string representation of the graph\n        \"\"\"\n        string = \"\"\n        for tail in self.adjacency:\n            for head in self.adjacency[tail]:\n                weight = self.adjacency[head][tail]\n                string += f\"{head} -> {tail} == {weight}\\n\"\n        return string.rstrip(\"\\n\")\n\n    def get_edges(self):\n        \"\"\"\n        Returna all edges in the graph\n        \"\"\"\n        output = []\n        for tail in self.adjacency:\n            for head in self.adjacency[tail]:\n                output.append((tail, head, self.adjacency[head][tail]))\n        return output\n\n    def get_vertices(self):\n        \"\"\"\n        Returns all vertices in the graph\n        \"\"\"\n        return self.adjacency.keys()\n\n    @staticmethod\n    def build(vertices=None, edges=None):\n        \"\"\"\n        Builds a graph from the given set of vertices and edges\n\n        \"\"\"\n        g = Graph()\n        if vertices is None:\n            vertices = []\n        if edges is None:\n            edge = []\n        for vertex in vertices:\n            g.add_vertex(vertex)\n        for edge in edges:\n            g.add_edge(*edge)\n        return g\n\n    class UnionFind:\n        \"\"\"\n        Disjoint set Union and Find for Boruvka's algorithm\n        \"\"\"\n\n        def __init__(self):\n            self.parent = {}\n            self.rank = {}\n\n        def __len__(self):\n            return len(self.parent)\n\n        def make_set(self, item):\n            if item in self.parent:\n                return self.find(item)\n\n            self.parent[item] = item\n            self.rank[item] = 0\n            return item\n\n        def find(self, item):\n            if item not in self.parent:\n                return self.make_set(item)\n            if item != self.parent[item]:\n                self.parent[item] = self.find(self.parent[item])\n            return self.parent[item]\n\n        def union(self, item1, item2):\n            root1 = self.find(item1)\n            root2 = self.find(item2)\n\n            if root1 == root2:\n                return root1\n\n            if self.rank[root1] > self.rank[root2]:\n                self.parent[root2] = root1\n                return root1\n\n            if self.rank[root1] < self.rank[root2]:\n                self.parent[root1] = root2\n                return root2\n\n            if self.rank[root1] == self.rank[root2]:\n                self.rank[root1] += 1\n                self.parent[root2] = root1\n                return root1\n            return None\n\n    @staticmethod\n    def boruvka_mst(graph):\n        \"\"\"\n        Implementation of Boruvka's algorithm\n        >>> g = Graph()\n        >>> g = Graph.build([0, 1, 2, 3], [[0, 1, 1], [0, 2, 1],[2, 3, 1]])\n        >>> g.distinct_weight()\n        >>> bg = Graph.boruvka_mst(g)\n        >>> print(bg)\n        1 -> 0 == 1\n        2 -> 0 == 2\n        0 -> 1 == 1\n        0 -> 2 == 2\n        3 -> 2 == 3\n        2 -> 3 == 3\n        \"\"\"\n        num_components = graph.num_vertices\n\n        union_find = Graph.UnionFind()\n        mst_edges = []\n        while num_components > 1:\n            cheap_edge = {}\n            for vertex in graph.get_vertices():\n                cheap_edge[vertex] = -1\n\n            edges = graph.get_edges()\n            for edge in edges:\n                head, tail, weight = edge\n                edges.remove((tail, head, weight))\n            for edge in edges:\n                head, tail, weight = edge\n                set1 = union_find.find(head)\n                set2 = union_find.find(tail)\n                if set1 != set2:\n                    if cheap_edge[set1] == -1 or cheap_edge[set1][2] > weight:\n                        cheap_edge[set1] = [head, tail, weight]\n\n                    if cheap_edge[set2] == -1 or cheap_edge[set2][2] > weight:\n                        cheap_edge[set2] = [head, tail, weight]\n            for vertex in cheap_edge:\n                if cheap_edge[vertex] != -1:\n                    head, tail, weight = cheap_edge[vertex]\n                    if union_find.find(head) != union_find.find(tail):\n                        union_find.union(head, tail)\n                        mst_edges.append(cheap_edge[vertex])\n                        num_components = num_components - 1\n        mst = Graph.build(edges=mst_edges)\n        return mst\n"
}

print(record["_source"])